In [ ]:
colab = False
try:
  import google.colab
  colab = True
except:
  import os
  os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
  os.environ["TORCH_MPS"] = "1"

print(f'colab mode: {colab}')
!pwd

In [ ]:
import torch
requirements_installed = False
if torch.__version__.startswith('2.'):
  requirements_installed = True
print(f'requirements_installed: {requirements_installed}')

In [ ]:
if colab:
  !git clone https://github.com/iSevenDays/nanoGPT.git
  path = !pwd
  if 'nanoGPT' not in path:
    %cd nanoGPT/
  !git pull origin master

In [ ]:
if colab:
  !nvidia-smi

In [ ]:
if colab and not requirements_installed:
    !pip install accelerate==0.15.0 \
    attrs==22.2.0 \
    bitsandbytes==0.35.4 \
    certifi==2022.12.7 \
    cffi==1.15.1 \
    charset-normalizer==3.1.0 \
    cython==0.29.33 \
    datasets==2.10.1 \
    dill==0.3.6 \
    happytransformer==2.4.1 \
    huggingface-hub==0.11.1 \
    git+https://github.com/cccntu/minLoRA.git@main \
    multiprocess==0.70.14 \
    mwparserfromhell==0.6.4 \
    nltk==3.8.1 \
    numpy \
    protobuf==3.19.5 \
    psutil==5.9.4 \
    python-dateutil==2.8.2 \
    python-json-logger==2.0.7 \
    pytorch-lightning==1.9.4 \
    requests==2.28.2 \
    responses==0.18.0 \
    sentencepiece==0.1.97 \
    six==1.16.0 \
    taming-transformers==0.0.1 \
    tiktoken \
    timm==0.4.12 \
    tokenizers==0.13.2 \
    torch --pre --force-reinstall --extra-index-url https://download.pytorch.org/whl/nightly/cu117 \
    torchmetrics==0.11.3 \
    tqdm==4.65.0 \
    transformers==4.26.1 \
    typing-extensions==4.5.0 \
    urllib3==1.26.14 \
    wandb \
    pathos \
    chardet



In [ ]:
# Click on restart runtime!!!

In [ ]:
from functools import partial

import torch
from minlora import LoRAParametrization

lora_dropout_p = 0.0
rank = 256
lora_alpha = 256
lora_config = {
    torch.nn.Embedding: {
        "weight": partial(LoRAParametrization.from_embedding, rank=rank, lora_alpha=lora_alpha),
    },
    torch.nn.Linear: {
        "weight": partial(LoRAParametrization.from_linear, rank=rank, lora_alpha=lora_alpha),
    },
}

In [ ]:


"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import math
import pickle
import inspect
from contextlib import nullcontext

import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

import minlora
# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 20
log_interval = 1
eval_iters = 100 # to calculate loss
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
block_size = 256
# wandb logging
wandb_log = True # disabled by default
use_lora = False
# model
n_layer = 16
n_head = 20
n_embd = 320
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?


#### RWKV specific
# model
ctx_len = 256
rwkv_emb_scale = 0.4
rwkv_tiny_attn = 2
rwkv_tiny_head = 2
n_attn = n_embd
n_ffn = n_embd
### RWKV END

from enum import Enum

class ModelType(Enum):
    ChatGPT = 1
    Lama = 2
    Hyena = 3
    RWKV = 4

model_type = ModelType.RWKV

wandb_project = f'united_medium'

if model_type == ModelType.Hyena:
    if use_lora:
        checkpoint_name = f'united_medium_{block_size}-gpthyena-lora-{rank}_{lora_alpha}_nl{n_layer}_ne{n_embd}.pt'
        wandb_run_name = f'gpt2-hyena-lora-{rank}_{lora_alpha}_nl{n_layer}_ne{n_embd}'
    else:
        checkpoint_name = f'united_medium_{block_size}-gpthyena-adam_nl{n_layer}_ne{n_embd}.pt'
        wandb_run_name = f'gpt2-hyena-adam_nl{n_layer}_ne{n_embd}'
elif model_type == ModelType.RWKV:
    if use_lora:
        checkpoint_name = f'united_medium_{block_size}-gptrwkv-lora-{rank}_{lora_alpha}_nl{n_layer}_ne{n_embd}.pt'
        wandb_run_name = f'gpt2-rwkv-lora-{rank}_{lora_alpha}_nl{n_layer}_ne{n_embd}_attn{rwkv_tiny_attn}'
    else:
        checkpoint_name = f'united_medium_{block_size}-gptrwkv-adam_nl{n_layer}_ne{n_embd}.pt'
        wandb_run_name = f'gpt2-rwkv-adam_nl{n_layer}_ne{n_embd}'
elif model_type == ModelType.Lama:
    wandb_project = f'united_medium'
    if use_lora:
        checkpoint_name = f'united_medium_{block_size}-gptlama-lora-{rank}_{lora_alpha}_nl{n_layer}_ne{n_embd}.pt'
        wandb_run_name = f'gpt2-lama-lora-{rank}_{lora_alpha}_nl{n_layer}_ne{n_embd}'
    else:
        checkpoint_name = f'united_medium_{block_size}-gptlama-adam_nl{n_layer}_ne{n_embd}.pt'
        wandb_run_name = f'gpt2-lama-adam_nl{n_layer}_ne{n_embd}'
elif model_type == ModelType.ChatGPT:
    checkpoint_name = f'united_medium_{block_size}-gpt.pt'
    wandb_run_name = f'gpt2_nl{n_layer}_ne{n_embd}'

if model_type == ModelType.RWKV:
    from rwkv import GPT, GPTConfig
else:
    from model import GPTConfig, GPT
# data
dataset = 'united'
gradient_accumulation_steps = 5 # used to simulate larger batch sizes
batch_size = 14#24 # ~8 for A100; if gradient_accumulation_steps > 1, this is the micro-batch size

# adamw optimizer
learning_rate = 1e-3 #1e-3 for lora #6e-4 # max learning rate
max_iters = 10000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 20 # how many steps to warm up for
lr_decay_iters = 10000 # should be ~= max_iters per Chinchilla
min_lr = 9e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
 # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'float16' if colab else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = False # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
#exec(open('configurator.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    print('Will use ddp')
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
else:
    print('Will not use ddp')
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    gradient_accumulation_steps *= 8 # simulate 8 gpus

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
if colab:
    torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
    torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [ ]:
assert n_embd % n_head == 0

In [ ]:
data_dir = os.path.join('data', dataset)

In [ ]:
if colab:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
if colab:
  !ls /content/drive/MyDrive/gpt-checkpoints
  out_dir = '/content/drive/MyDrive/gpt-checkpoints'


In [ ]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9
tokens_seen = 0

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
if model_type == ModelType.RWKV:
    vocab_size = meta_vocab_size if meta_vocab_size is not None else 50304
    model_args = dict(vocab_size=vocab_size, ctx_len=ctx_len, model_type="RWKV",
                  rwkv_emb_scale=rwkv_emb_scale, rwkv_tiny_attn=rwkv_tiny_attn, rwkv_tiny_head=rwkv_tiny_head,
                  n_layer=n_layer, n_head=n_head, n_embd=n_embd, n_attn=n_attn,
                  n_ffn=n_ffn)
else:
    model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout, hyena=model_type==ModelType.Hyena,
                  rotary=model_type==ModelType.Lama) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    # determine the vocab size we'll use for from-scratch training
    if meta_vocab_size is None:
        print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
    model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, checkpoint_name)
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        try:
            val = checkpoint_model_args[k]
        except KeyError:
            continue
        model_args[k] = val
        if k == 'n_layer' and n_layer != val:
          print(f'Override {k} from {n_layer} to {val}')
          n_layer = val
        if k == 'n_head' and n_head != val:
          print(f'Override {k} from {n_head} to {val}')
          n_head = val
        if k == 'n_embd' and n_embd != val:
          print(f'Override {k} from {n_embd} to {val}')
          n_embd = val
    # create the model
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    if use_lora:
        minlora.add_lora(model, lora_config)

    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

    model.load_state_dict(state_dict)
    if use_lora:
        # the full state dict includes the LoRA state dict
        # so actually we don't need to load it separately again
        model.load_state_dict(checkpoint['lora'], strict=False)
        print('Loaded LoRA state dict')
        # sanity check
        #model.apply(minlora.apply_to_lora(lambda m: print((m.lora_A.sum(), m.lora_B.sum()))))
        # merge for zero-overhead inference
        minlora.merge_lora(model)

    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
    tokens_seen = checkpoint.get('tokens_seen', 0)

    if tokens_seen == 0:
        tokens_seen = batch_size * gradient_accumulation_steps * n_embd
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model = GPT.from_pretrained(init_from, override_args)
    # read off the created config params, so we can store them into checkpoint correctly
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = getattr(model.config, k)
# crop down the model block size if desired, using model surgery
if model_type != ModelType.RWKV:
    if block_size < model.config.block_size:
        model.crop_block_size(block_size)
        model_args['block_size'] = block_size # so that the checkpoint will have the right value
if use_lora:
    print("Using lora")
    minlora.add_lora(model, lora_config=lora_config)
    if model_type == ModelType.RWKV:
        minlora.tie_weights(linear=model.head, embedding=model.tok_emb)
    else:
        minlora.tie_weights(linear=model.lm_head, embedding=model.transformer.wte)
model.to(device)

In [ ]:
#!python -m spacy download en_core_web_sm

In [ ]:
import importlib
import CachedDataLoader
importlib.reload(CachedDataLoader)

In [ ]:
MAX_LENGTH = block_size

In [ ]:
from CachedDataLoader import CachedDataLoader
import time
from transformers import GPT2Tokenizer
import torch

#
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
BUFFER_SIZE = 10000
#
MAX_LENGTH = block_size
parallel = True
use_pile = False
# Define a function to download and preprocess the data in a background thread

dataset_weights = {
    'pile': 0.6,
    'wiki': 0.1,
    'bookcorpus': 0.3
} if use_pile else {'wiki':0.1, 'bookcorpus': 0.9}

datasets=['wiki', 'pile'] if use_pile else ['wiki', 'bookcorpus']

data_loader = CachedDataLoader(datasets, dataset_weights)

In [ ]:
from datasets import load_dataset
#cache_dir = '/Volumes/WD_RED_RW/machine_learning/cache'
#cache_dir = '/Volumes/homes/seven/Btrfs_main_backup/Macbook14M1Leipzig.local/Volumes/WD_RED_RW/machine_learning/cache'
dataset = load_dataset("wikitext", 'wikitext-103-v1', split="train")
#dataset = load_dataset("EleutherAI/the_pile_deduplicated", split="train", cache_dir=cache_dir)

In [ ]:
shard = dataset.shard(num_shards=128, index=0)
shard

In [ ]:
split = shard.train_test_split(test_size=0.1, seed=42)

In [ ]:
train_dataset = split['train']
test_dataset = split['test']
split['train'][0]

In [ ]:
# cache_dir = '/Users/seven/cache'
# from datasets import load_from_disk
# train_cache_file = f"{cache_dir}/train_split.arrow"
# test_cache_file = f"{cache_dir}/test_split.arrow"
# if os.path.exists(train_cache_file):
#     train_dataset = load_from_disk(train_cache_file)
# else:
#     split['train'].save_to_disk(train_cache_file)
#
# if os.path.exists(test_cache_file):
#     test_dataset = load_from_disk(test_cache_file)
# else:
#     split['test'].save_to_disk(test_cache_file)

In [ ]:
import random
def randomly_select_element(dataset):
    num_rows = len(dataset)
    random_index = random.randint(0, num_rows - 1)
    random_element = dataset[random_index]
    return random_element

randomly_select_element(train_dataset)

In [ ]:
from IrrelevantChecker import IrrelevantChecker
import importlib
import text_cleaner
importlib.reload(text_cleaner)
from text_cleaner import TextCleaner, TextCleaningException

textcleaner = TextCleaner()

from is_irrelevant import is_irrelevant
relevance_checker = IrrelevantChecker(is_irrelevant)

In [ ]:
def randomly_select_valid_clean_element(dataset, debug=False):
    result_text = None
    original_text = None
    while result_text is None:
        element = randomly_select_element(dataset)
        try:
            original_text = element['text']
            result_text = textcleaner.clean(original_text)
            if not relevance_checker.is_relevant(result_text):
                if debug:
                    print(f'Skipping not relevant: {result_text}')
                result_text = None
        except TextCleaningException as e:
            if e.error_reason is text_cleaner.CleaningError.INSUFFICIENT_SENTENCES:
                if debug:
                    print(f"Text cleaning failed: {e}, cleaned_text: {e.cleaned_text}, original_text: {element['text']}")
            else:
                if debug:
                    print(f"Text cleaning failed: {e}")
    if debug:
        print(f"Text cleaning finished, original text:\n{original_text}")
    return result_text

# Example usage
random_element = randomly_select_valid_clean_element(train_dataset, debug=True)
print('Found text')
print(random_element)

In [ ]:
def get_train_item_slow():
    text = randomly_select_valid_clean_element(train_dataset)
    ids = tokenizer.encode(text, truncation=True)
    ids.append(tokenizer.eos_token_id)
    return ids

def get_validation_item_slow():
    text = randomly_select_valid_clean_element(test_dataset)
    ids = tokenizer.encode(text, truncation=True)
    ids.append(tokenizer.eos_token_id)
    return ids

In [ ]:
from queue import Queue
from concurrent.futures import ThreadPoolExecutor

queue_train = Queue(maxsize=1000)  # Adjust the maxsize to control the maximum number of prefetched items
queue_validation = Queue(maxsize=1000)

def prefetch_train_items(queue_train):
    while True:
        item = get_train_item_slow()
        queue_train.put(item)

def prefetch_validation_items(queue_validation):
    while True:
        item = get_validation_item_slow()
        queue_validation.put(item)

# Create the ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=2)

# Start the prefetching threads
executor.submit(prefetch_train_items, queue_train)
executor.submit(prefetch_validation_items, queue_validation)

print('started')

# You can execute other code here without waiting for the threads


In [ ]:
use_data_loader = True
def get_train_item():
    if use_data_loader:
        return data_loader.get_train_item()
    try:
        return queue_train.get(timeout=2)
    except:
        print('Getting slow item train')
        return get_train_item_slow()

    #return get_train_item_slow()

def get_validation_item():
    if use_data_loader:
        return data_loader.get_validation_item()
    try:
        return queue_validation.get(timeout=2)
    except:
        print('Getting slow item validation')
        return get_validation_item_slow()
    #return get_validation_item_slow()

In [ ]:
def get_train_queue_size():
    if use_data_loader:
        return data_loader.get_train_queue_size()
    return queue_train.qsize()

def get_validation_queue_size():
    if use_data_loader:
        return data_loader.get_validation_queue_size()
    return queue_validation.qsize()

In [ ]:
def get_batch(split, print_prompt_target_texts: bool = False, print_queue_change = True):
    buffer_ids = []
    result_ids = []
    # if split == 'train':
    #     queue_size_old = get_train_queue_size()
    # else:
    #     queue_size_old = get_validation_queue_size()
    i = 0
    global data_loader
    while True:
        if len(result_ids) == batch_size:
            break
        if split == 'train':
            ids = get_train_item()
        else:
            ids = get_validation_item()

        buffer_ids.extend(ids)

        while len(buffer_ids) > MAX_LENGTH + 1:
            if len(result_ids) == batch_size:
                break
            # Truncate text if it's longer than MAX_LENGTH
            id = buffer_ids[:MAX_LENGTH + 1]
            result_ids.append(id)
            buffer_ids = buffer_ids[MAX_LENGTH:]

        i = i + 1

    # if split == 'train':
    #     queue_size = get_train_queue_size()
    # else:
    #     queue_size = get_validation_queue_size()
    #diff = queue_size_old - queue_size
    #print(f'Split {split} took: {diff} elements for {len(result_ids)} elements of {MAX_LENGTH} size (of {queue_size} left)')
    x = torch.stack([torch.tensor(ids[:-1]) for ids in result_ids])
    y = torch.stack([torch.tensor(ids[1:]) for ids in result_ids])

    if print_prompt_target_texts:
        # Decode x back to text
        prompt_text = [tokenizer.batch_decode(ids) for ids in x]
        target_text = [tokenizer.batch_decode(ids) for ids in y]

        # Print prompts
        for i in range(len(prompt_text)):
            print(f"Prompt {i + 1}: {''.join(prompt_text[i][:60])}")
            print(f"Target {i + 1}: {''.join(target_text[i][:60])}")

    if device == 'cuda':
        # Pin arrays x, y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

#get_batch('train')
#get_batch('train')
x, y = get_batch('train', print_prompt_target_texts = True)
# x, y = get_batch('train', print_prompt_target_texts = True)
# x, y = get_batch('train', print_prompt_target_texts = True)
# x, y = get_batch('train', print_prompt_target_texts = True)
# x, y = get_batch('train', print_prompt_target_texts = True)
# x, y = get_batch('train', print_prompt_target_texts = True)
print(f'x shape: {x.shape}')  # should print "x shape: torch.Size([12, 1024])"
print(f'y shape: {y.shape}')  # should print "y shape: torch.Size([12, 1024])"

In [ ]:
import google.protobuf
print(google.protobuf.__version__)

In [ ]:
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
def configure_optimizers_lora(self, weight_decay, learning_rate, betas, device_type):
    # we apply weight decay to all lora params
    optim_groups = [
        # note: .get_lora_params() returns a generator
        # we need to wrap it in a list so we can consume it twice
        {"params": list(minlora.get_lora_params(self)) , "weight_decay": weight_decay},
        # you can also add biases for fine-tuning,
        # but I want to make sure lora alone works
        # {"params": minlora.get_bias_params(self), "weight_decay": 0.0}, # bias params don't get weight decay
    ]

    def parameter_count(optim_groups):
        n = sum(p.numel() for group in optim_groups for p in group["params"])
        if n < 1e6:
            return f"{n/1e3:.1f}k"
        else:
            return f"{n/1e6:.1f}M"

    print(f"optimizing {parameter_count(optim_groups)} parameters")

    # new PyTorch nightly has a new 'fused' option for AdamW that is much faster
    use_fused = (device_type == "cuda") and ("fused" in inspect.signature(torch.optim.AdamW).parameters)
    print(f"using fused AdamW: {use_fused}")
    extra_args = dict(fused=True) if use_fused else dict()
    optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)

    return optimizer
if use_lora:
    optimizer = configure_optimizers_lora(model, weight_decay, learning_rate, (beta1, beta2), device_type)
else:
    optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])

    # disable the fused option
    for group in optimizer.param_groups:
        if group['fused'] == True and device != 'cuda':
            print('Disabling previously enabled `fused` option')
        elif group['fused'] == False and device == 'cuda':
            print('Enabling previously disabled `fused` option')
        group['fused'] = device == 'cuda'

if device == 'mps':
    torch.backends.mps.enabled = True


In [ ]:
print(device)

In [ ]:

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    try:
        if wandb_init_completed:
            print('wand already initialized')
    except:
        wandb_init_completed = True
        import wandb

        if colab:
            
            wandb.login(key=wb_key)

        wandb.init(project=wandb_project, name=wandb_run_name, config=config)


In [ ]:
# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model.module if ddp else model # unwrap DDP container if needed
running_mfu = -1.0

In [ ]:
raw_model

In [ ]:
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
                'tokens_seen': tokens_seen,
            }, step=int(iter_num/eval_interval))
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'tokens_seen': tokens_seen,
                    'config': config,
                }
                if use_lora:
                    checkpoint['lora'] = minlora.get_lora_state_dict(raw_model)
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, checkpoint_name))
    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        if ddp:
            # in DDP training we only need to sync gradients at the last micro step.
            # the official way to do this is with model.no_sync() context manager, but
            # I really dislike that this bloats the code and forces us to repeat code
            # looking at the source of that context manager, it just toggles this variable
            model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
        with ctx:
            logits, loss = model(X, Y)
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        #print(f'Will get x, y')
        X, Y = get_batch('train')
        # backward pass, with gradient scaling if training in fp16
        #print(f'Will scale and backward {micro_step} from {gradient_accumulation_steps}')
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        lossf = loss.item() # loss as float. note: this is a CPU-GPU sync point
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1
    tokens_seen += batch_size * gradient_accumulation_steps * n_embd

    # termination conditions
    if iter_num > max_iters:
        break

if ddp:
    destroy_process_group()